In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import libry as ry
import numpy as np
import time

from skimage.measure import compare_ssim

In [2]:
#-- Add REAL WORLD configuration and camera
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)

#RealWorld.getFrameNames()

#Change color of objects
RealWorld.frame("obj1").setColor([1.,.0,.0])
RealWorld.frame("obj13").setColor([1.,.0,.0])
RealWorld.frame("obj2").setColor([1.,.0,.0])
RealWorld.frame("obj23").setColor([1.,.0,.0])
RealWorld.frame("obj19").setColor([1.,.0,.0])
S.addSensor("camera")


C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")


In [3]:
#the focal length
#f = 0.895
f = 0.7
f = f * 360.
#the relative pose of the camera
# pcl.setRelativePose('d(-90 0 0 1) t(-.08 .205 .115) d(26 1 0 0) d(-1 0 1 0) d(6 0 0 1) ')
fxfypxpy = [f, f, 320., 180.]

#### Exercise 1
1.) Filter the color of the image to find all pixels that are redish.
2.) Display the binary image that indicates red pixels.
3.) Fit contours to this binary image, which returns the segments.
4.) Display the contours by drawing them into the original RGB image

convexHull? Check out


In [ ]:
points = []
tau = .01

for t in range(300):
    time.sleep(0.03)
    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:

        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
        cameraFrame.setPointCloud(points, rgb)
        V.recopyMeshes(C)
        V.setConfiguration(C)

        #use hsv - simpler for color finding
        hsv = cv.cvtColor(rgb, cv.COLOR_BGR2HSV)

        lower_red = np.array([115, 100, 100])
        upper_red = np.array([130, 255, 255])

        mask = cv.inRange(hsv, lower_red, upper_red)

        #find contours adn draw in rgb image
        contours, hierarchy = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        cv.drawContours(rgb, contours, -1, (0, 255, 0), 3)

        if len(rgb)>0: cv.imshow('OPENCV - rgb', rgb)
        #if len(hsv)>0: cv.imshow('OPENCV - hsv', hsv)
        if len(mask)>0: cv.imshow('OPENCV - mask', mask)


        if cv.waitKey(1) & 0xFF == ord('q'):
            break


    S.step([], tau, ry.ControlMode.none)



#hsv = cv.cvtColor(cameraFrame, cv.COLOR_BGR2HSV)
#cv.imshow('HSV', hsv)


In [ ]:
cv.destroyAllWindows()

#### Exercise 2
1.) Store the first image as background image. (Or average over several first images.)
2.) For every new image filter those pixels, that are significantly different to the background.
3.) Display the binary image that indicates change pixels.
4.) Fit contours to this binary image, which returns the segments.
5.) Display the contours by drawing them into the original RGB image
6.) Test the same pipeline, but first smoothing/blurring the image


Extra - compare_ssim

The score
  represents the structural similarity index between the two input images. This value can fall into the range [-1, 1] with a value of one being a “perfect match”.

The diff
  image contains the actual image differences between the two input images that we wish to visualize. The difference image is currently represented as a floating point data type in the range [0, 1] so we first convert the array to 8-bit unsigned integers in the range [0, 255] (Line 26) before we can further process it using OpenCV.

In [4]:
[ref_rgb, ref_depth] = S.getImageAndDepth()

points = []
tau = .01

for t in range(500):
    time.sleep(0.1)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
        cameraFrame.setPointCloud(points, rgb)
        V.recopyMeshes(C)
        V.setConfiguration(C)

        #not sure why?
        #ref_rgb = cv.blur(ref_rgb,(5,5))
        #ref_rgb = cv.GaussianBlur(ref_rgb,(5,5),0)
        #rgb = cv.GaussianBlur(rgb,(5,5),0)


        # Convert images to grayscale
        ref_rgb_gray = cv.cvtColor(ref_rgb, cv.COLOR_BGR2GRAY)
        rgb_gray = cv.cvtColor(rgb, cv.COLOR_BGR2GRAY)

        # Compute SSIM between two images
        (score, diff) = compare_ssim(ref_rgb_gray, rgb_gray, full=True)
        diff = (diff * 255).astype("uint8")
        print("SSIM: {}".format(score))

        thresh = cv.threshold(diff, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
        contours, __ = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0: cv.drawContours(rgb, contours, -1, (0, 255, 0), 1)

        if len(rgb)>0: cv.imshow('OPENCV - rgb', rgb)
        if len(depth)>0: cv.imshow('OPENCV - diff', diff)
        if len(depth)>0: cv.imshow('OPENCV - Threshold', thresh)

        [ref_rgb, ref_depth] = S.getImageAndDepth()
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    S.step([], tau, ry.ControlMode.none)


/home/nika/.local/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.


SSIM: 1.0
SSIM: 0.9145864963243804
SSIM: 0.8708617628128281
SSIM: 0.8354997643390504
SSIM: 0.7867769965631851
SSIM: 0.7680874534436792
SSIM: 0.7736531145853235
SSIM: 0.8268022636690561
SSIM: 0.8254909459814985
SSIM: 0.8201170242046291
SSIM: 0.8305374215688
SSIM: 0.8603625542976248
SSIM: 0.8591972029020896
SSIM: 0.8833335264891294
SSIM: 0.9227681185622427
SSIM: 0.9623136416475
SSIM: 0.9497003168470564
SSIM: 0.94917242134912
SSIM: 0.9727469186395067
SSIM: 0.959109122856326
SSIM: 0.9643392837185566
SSIM: 0.9774839873200998
SSIM: 0.9920056877216228
SSIM: 0.979274061500213
SSIM: 0.974684764212507
SSIM: 0.9796662986135392
SSIM: 0.9836855495229825
SSIM: 0.9736877091517989
SSIM: 0.9763850126964138
SSIM: 0.9802497749127673
SSIM: 0.9884421034146312
SSIM: 0.9942123283809479
SSIM: 0.986388566429694
SSIM: 0.9799009466321782
SSIM: 0.9779691158716421
SSIM: 0.9899201217976052
SSIM: 0.9834553509680252
SSIM: 0.9794029965539252
SSIM: 0.98624773311601
SSIM: 0.992737418222676
SSIM: 0.981649154155768
SSIM: 

#### Exercise 3
Exercise 2 is doing foreground/background segmentation based on the RGB.

Think about how one can do the same for depth.
In particular, assume that ‘background’ has always the largest pixel depth – so when the depth of a pixel is less than before,
then it must be foreground. Realize this within the simulation loop, where you have access to depth.



In [5]:
points = []
tau = .01

for t in range(300):
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
        cameraFrame.setPointCloud(points, rgb)
        V.recopyMeshes(C)
        V.setConfiguration(C)

        rgb_gray = cv.cvtColor(rgb, cv.COLOR_BGR2GRAY)
        thresh = cv.threshold(rgb_gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)[1]

        cv.normalize(depth, depth,  0, 255, cv.NORM_MINMAX)
        #thresh_depth = cv.threshold(depth, .4, .2, cv.THRESH_BINARY)[1]


        #kernel = np.ones((3,3),np.uint8)
        #dilation = cv.dilate(depth, kernel, depth)

        if len(rgb)>0: cv.imshow('OPENCV - rgb', rgb)
        if len(depth)>0: cv.imshow('OPENCV - dilation', depth)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    S.step([], tau, ry.ControlMode.none)
    


In [ ]:

cv.destroyAllWindows()
